In [ ]:
# Takes the output from routing_3.ipynb and works on the routing portion of it
import polars as pl
import numpy as np
import networkx as nx
from rt_bundled_ego_chord_diagram import RTBundledEgoChordDiagram
import rtsvg
rt = rtsvg.RACETrack()

edges_filename  = '../../data/stanford/facebook/1684.edges'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = _split_[0], _split_[1]
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df = pl.DataFrame(_lu_)
# Only Keep The Largest Component
g = rt.createNetworkXGraph(df, [('fm','to')])
comp_largest = None
for _comp_ in nx.connected_components(g):
    if comp_largest is None:            comp_largest = _comp_
    if len(_comp_) > len(comp_largest): comp_largest = _comp_
df = df.filter(pl.col('fm').is_in(comp_largest) & pl.col('to').is_in(comp_largest))
_relates_ = [('fm','to')]
'''
import os
_dir_   = '../../data/2013_vast_challenge/mc3_netflow/nf'
_files_ = os.listdir(_dir_)
_dfs_   = []
for _file_ in _files_:
    _dfs_.append(pl.read_csv(_dir_ + '/' + _file_))
    break
df = pl.concat(_dfs_) # .sample(50_000)
_relates_ = [('firstSeenSrcIp', 'firstSeenDestIp')]
'''
df.shape, len(set(df['fm']) | set(df['to']))

In [ ]:
becd = RTBundledEgoChordDiagram(rt, df, _relates_, high_degree_node_count=25, chord_diagram_points=2, w=1500, h=900)
# rt.link(becd.df_routing, [('__fm__','__to__')], becd.pos_routing, node_size='small', w=1500, h=900)
becd

In [ ]:
rt.tile([becd.outlineSVG(render_inter_edges=False)])

In [ ]:
def pivot(_lu_):
    df_lu = {'bin':[],'__count__':[]}
    for k in _lu_: df_lu['bin'].append(k), df_lu['__count__'].append(_lu_[k])
    return pl.DataFrame(df_lu)
rt.histogram(pivot(becd.time_lu), bin_by='bin', count_by='__count__', w=512)

In [ ]:
'''
_tiles_ = []
for _fm_ in set(becd.df_aligned_btwn_communities_collapsed['__fm__']):
    _max_ = 1
    svgs = [f'<svg x="0" y="0" width="{becd.w}" height="{becd.h}"><rect x="0" y="0" width="{becd.w}" height="{becd.h}" fill="#ffffff"/>']
    for _segment_ in becd.segment_contains_tree:
        _xy0_, _xy1_ = becd.pos_routing[_segment_[0]], becd.pos_routing[_segment_[1]]
        svgs.append(f'<line x1="{_xy0_[0]}" y1="{_xy0_[1]}" x2="{_xy1_[0]}" y2="{_xy1_[1]}" stroke="#ff0000" stroke-width="1.0"/>')
        if _fm_ in becd.segment_contains_tree[_segment_]:
            if becd.segment_contains_tree[_segment_][_fm_] > _max_:
                _max_ = becd.segment_contains_tree[_segment_][_fm_]
    for _segment_ in becd.segment_contains_tree:
        if _fm_ in becd.segment_contains_tree[_segment_]:
            _xy0_, _xy1_ = becd.pos_routing[_segment_[0]], becd.pos_routing[_segment_[1]]
            _width_ = 5.0 * becd.segment_contains_tree[_segment_][_fm_] / _max_
            svgs.append(f'<line x1="{_xy0_[0]}" y1="{_xy0_[1]}" x2="{_xy1_[0]}" y2="{_xy1_[1]}" stroke="#000000" stroke-width="{_width_}"/>')
    svgs.append(rt.svgText(_fm_, 5, becd.h - 5))
    svgs.append('</svg>')
    _tiles_.append(''.join(svgs))
rt.tile(_tiles_, horz=False, spacer=10)
'''
# Find the most frequent occuring _fm_
_sorter_ = []
for _fm_ in set(becd.df_aligned_btwn_communities_collapsed['__fm__']):
    _count_, _max_ = 0, 1
    for _segment_ in becd.segment_contains_tree:
        if _fm_ in becd.segment_contains_tree[_segment_]:
            _count_ += 1
            _max_    = max(_max_, becd.segment_contains_tree[_segment_][_fm_])
    _sorter_.append((_count_, _fm_, _max_))
_sorter_ = sorted(_sorter_, reverse=True)
_count_, _fm_, _max_ = _sorter_[0]
svgs = [f'<svg x="0" y="0" width="{becd.w}" height="{becd.h}"><rect x="0" y="0" width="{becd.w}" height="{becd.h}" fill="#ffffff"/>']
for _segment_ in becd.segment_contains_tree:
    if _fm_ in becd.segment_contains_tree[_segment_]:
        _xy0_, _xy1_ = becd.pos_routing[_segment_[0]], becd.pos_routing[_segment_[1]]
        _width_ = 1.0 + 9.0 * becd.segment_contains_tree[_segment_][_fm_] / _max_
        svgs.append(f'<line x1="{_xy0_[0]}" y1="{_xy0_[1]}" x2="{_xy1_[0]}" y2="{_xy1_[1]}" stroke="#000000" stroke-width="{_width_}"/>')
_x_, _y_ = becd.pos_routing[_fm_]
svgs.append(f'<circle cx="{_x_}" cy="{_y_}" r="6" fill="none" stroke="#ff0000"/>')
for i in range(len(becd.circles)):
    _circle_ = becd.circles[i]
    svgs.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="none" stroke="#d0d0d0"/>')

svgs.append('</svg>')
print(_fm_)
print('goes to', len(set(becd.df_aligned_btwn_communities_collapsed.filter(pl.col('__fm__')==_fm_)['__to__'])))
rt.tile([''.join(svgs)])